# Ejercicio 3 - Splines Cúbicos
## Ian Oñate y Pablo Pacheco

In [2]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

def cubic_spline(xs, ys, B0, B1):
    points = sorted(zip(xs, ys), key=lambda x: x[0])
    xs = [x for x, _ in points]
    ys = [y for _, y in points]
    n = len(points) - 1
    h = [xs[i+1] - xs[i] for i in range(n)]

    alpha = [0] * (n + 1)
    alpha[0] = 3 * (ys[1] - ys[0]) / h[0] - 3 * B0
    alpha[n] = 3 * B1 - 3 * (ys[n] - ys[n - 1]) / h[n - 1]
    for i in range(1, n):
        alpha[i] = 3 / h[i] * (ys[i+1] - ys[i]) - 3 / h[i-1] * (ys[i] - ys[i-1])

    l = [1] + [0] * n
    u = [0] * n
    z = [0] * (n + 1)
    for i in range(1, n):
        l[i] = 2 * (xs[i+1] - xs[i-1]) - h[i-1] * u[i-1]
        u[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i-1] * z[i-1]) / l[i]
    l[n] = h[n - 1] * (2 - u[n - 1])
    z[n] = (alpha[n] - h[n - 1] * z[n - 1]) / l[n]

    c = [0] * (n + 1)
    b = [0] * n
    d = [0] * n
    a = [ys[i] for i in range(n)]

    c[n] = z[n]
    for j in range(n-1, -1, -1):
        c[j] = z[j] - u[j] * c[j+1]
        b[j] = (ys[j+1] - ys[j]) / h[j] - h[j] * (c[j+1] + 2 * c[j]) / 3
        d[j] = (c[j+1] - c[j]) / (3 * h[j])

    x = sym.Symbol('x')
    splines = []
    for j in range(n):
        S = a[j] + b[j]*(x - xs[j]) + c[j]*(x - xs[j])**2 + d[j]*(x - xs[j])**3
        splines.append(S)
    return splines

# DATOS EJERCICIO 3
xs = [0, 1, 2, 3]
ys = [-1, 1, 5, 2]
x_vals = np.linspace(xs[0], xs[-1], 500)
x = sym.Symbol('x')

fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2, label="Spline")
points_plot, = ax.plot(xs, ys, 'ro', label="Puntos")
tangent_line, = ax.plot([], [], 'g--', lw=1.5, label="Tangente")
angle_text = ax.text(0.7, 0.9, '', transform=ax.transAxes)
labels = [ax.text(0, 0, '', fontsize=9, ha='left', va='bottom') for _ in range(len(xs))]

ax.set_xlim(min(xs)-0.2, max(xs)+0.2)
ax.set_ylim(min(ys)-2, max(ys)+2)
ax.set_title("Spline cúbico al variar B₁")
ax.grid(True)
ax.legend(loc="upper left")

def animate(i):
    B1 = -5 + 0.2 * i
    splines = cubic_spline(xs, ys, B0=1, B1=B1)

    y_vals = []
    for xv in x_vals:
        for j in range(len(xs) - 1):
            if xs[j] <= xv <= xs[j+1]:
                y_vals.append(float(splines[j].subs(x, xv)))
                break
    line.set_data(x_vals, y_vals)

    x_last = xs[-1]
    y_last = ys[-1]
    dx = 0.3
    dy = B1 * dx
    tangent_line.set_data([x_last - dx, x_last + dx], [y_last - dy, y_last + dy])

    angle_deg = np.degrees(np.arctan(B1))
    angle_text.set_text(f"B₁ = {B1:.2f}\nθ ≈ {angle_deg:.1f}°")

    for idx, label in enumerate(labels):
        label.set_position((xs[idx] + 0.05, ys[idx] + 0.2))
        label.set_text(f"({xs[idx]}, {ys[idx]})")

    return line, points_plot, tangent_line, angle_text, *labels

anim = FuncAnimation(fig, animate, frames=51, interval=200)
anim.save("spline_B1_ejercicio3_final.gif", writer=PillowWriter(fps=5))
plt.close()
print("✅ GIF guardado como 'spline_B1_ejercicio3_final.gif'")

✅ GIF guardado como 'spline_B1_ejercicio3_final.gif'
